# Quantum State Tomography

* **Author:** Christopher J. Wood (cjwood@us.ibm.com)
* **Last Updated:** December 11, 2018

**Note:** This example should work with qiskit version 0.6 or 0.7 (qiskit-terra master branch)
For now it will print depreciation warnings when run on 0.7 but these will be changed once the stable version of 0.7 is officially released

**Note:** Sometimes the CVXOPT fitter fails

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.tools.qi.qi import state_fidelity

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 2-Qubit state tomography Example

In [2]:
# Create a state preparation circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell.qasm())

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

OPENQASM 2.0;
include "qelib1.inc";
qreg q0[2];
h q0[0];
cx q0[0],q0[1];

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [5]:
# Generate circuits and run on simulator
t = time.time()
qst_bell = tomo.state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

# Extract tomography data so that countns are indexed by measurement configuration
# Note that the None labels are because this is state tomography instead of process tomography
# Process tomography would have the preparation state labels there

tomo_counts_bell = tomo.tomography_data(job.result(), qst_bell)
tomo_counts_bell

Time taken: 0.7298009395599365


{('X', 'X'): {'00': 2512, '11': 2488},
 ('X', 'Y'): {'00': 1256, '01': 1248, '10': 1238, '11': 1258},
 ('X', 'Z'): {'00': 1222, '01': 1299, '10': 1242, '11': 1237},
 ('Y', 'X'): {'00': 1204, '01': 1281, '10': 1258, '11': 1257},
 ('Y', 'Y'): {'01': 2447, '10': 2553},
 ('Y', 'Z'): {'00': 1238, '01': 1266, '10': 1221, '11': 1275},
 ('Z', 'X'): {'00': 1240, '01': 1241, '10': 1246, '11': 1273},
 ('Z', 'Y'): {'00': 1250, '01': 1253, '10': 1279, '11': 1218},
 ('Z', 'Z'): {'00': 2500, '11': 2500}}

In [6]:
# Generate fitter data and reconstruct density matrix
probs_bell, basis_matrix_bell = tomo.fitter_data(tomo_counts_bell)
rho_bell = tomo.state_cvx_fit(probs_bell, basis_matrix_bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity =', F_bell)

Fit Fidelity = 0.999959328827773


## Including Calibration Error (Simulated)

Suppose we include T1 relaxation during measurement, but all other circuit operations are perfect.
Noise parameters:
* measurement time: 5 microseconds
* T_1: 50 microseconds
* Population of excited state: 5%

The assignment fidelity for this can be calculated from
* P(0|0) = 1 - P(1|0)
* P(1|0) = pop1 * exp(- t_meas / t_1)
* P(0|1) = (1 - pop1) * exp(- t_meas / t_1)
* P(1|1) = 1 - P(0|1)

the calibration error matrix is
M = [[P(0|0), P(0|1)], [P(1|0), P(1|1)]]

In [7]:
pop1 = 0.05
T1 = 50
t_meas = 5

noise = {
    'relaxation_rate': 1. / T1,
    'thermal_populations': [1 - pop1, pop1],
    'measure': {'gate_time': t_meas}
}

# Generate circuits and run on simulator
job_cal = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000,
                         config={'noise_params': noise})

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_noise = tomo.tomography_data(job_cal.result(), qst_bell)
tomo_counts_noise

{('X', 'X'): {'00': 2511, '01': 204, '10': 211, '11': 2074},
 ('X', 'Y'): {'00': 1488, '01': 1210, '10': 1226, '11': 1076},
 ('X', 'Z'): {'00': 1527, '01': 1262, '10': 1189, '11': 1022},
 ('Y', 'X'): {'00': 1551, '01': 1205, '10': 1244, '11': 1000},
 ('Y', 'Y'): {'00': 435, '01': 2282, '10': 2258, '11': 25},
 ('Y', 'Z'): {'00': 1517, '01': 1220, '10': 1207, '11': 1056},
 ('Z', 'X'): {'00': 1497, '01': 1200, '10': 1232, '11': 1071},
 ('Z', 'Y'): {'00': 1469, '01': 1231, '10': 1246, '11': 1054},
 ('Z', 'Z'): {'00': 2518, '01': 225, '10': 209, '11': 2048}}

In [8]:
# Create Calibration matrix

P1_0 = pop1 * np.exp(-t_meas / T1)
P0_1 = (1 - pop1) * np.exp(-t_meas / T1)

# Single qubit calibration matrix
cal_mat_single = np.array([[1 - P1_0, P0_1], [P1_0, 1 - P0_1]])

# 2-qubit calibration matrix
cal_mat = np.kron(cal_mat_single, cal_mat_single)

In [9]:
# Fitting without calibration matrix
probs_noise, basis_matrix_noise = tomo.fitter_data(tomo_counts_noise)
rho_noise = tomo.state_cvx_fit(probs_noise, basis_matrix_noise)
print('Fitting without using calibration data')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise))

Fitting without using calibration data
Fit Fidelity = 0.8678989336195806


In [10]:
# Fitting with calibration matrix by modifying POVMs
probs_noise_cal, basis_matrix_noise_cal = tomo.fitter_data(tomo_counts_noise,
                                                     calibration_matrix=cal_mat)
rho_noise_cal = tomo.state_cvx_fit(probs_noise_cal, basis_matrix_noise_cal)
print('Using calibration data to update basis matrix')
print('Fit Fidelity =', state_fidelity(psi_bell, rho_noise_cal))

Using calibration data to update basis matrix
Fit Fidelity = 0.9817596803651094


## Generating and fitting random states

We now test the functions on state generated by a circuit consiting of a layer of random single qubit unitaries u3

In [11]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
    psi_rand = job.result().get_statevector(circ)
    
    qst_circs = tomo.state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                         shots=shots)
    tomo_counts = tomo.tomography_data(job.result(), qst_circs)
    probs, basis_matrix = tomo.fitter_data(tomo_counts, beta=0.5)
    
    rho_cvx = tomo.state_cvx_fit(probs, basis_matrix)
    rho_mle = tomo.state_mle_fit(probs, basis_matrix)
    
    print('F fit (CVX) =', state_fidelity(psi_rand, rho_cvx))
    print('F fit (MLE) =', state_fidelity(psi_rand, rho_mle))

In [16]:
for j in range(5):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 5000)

Random single-qubit unitaries: set 0
F fit (CVX) = 0.9990365780552675
F fit (MLE) = 0.9974224179383847
Random single-qubit unitaries: set 1
F fit (CVX) = 0.999254906783349
F fit (MLE) = 0.9964909982409936
Random single-qubit unitaries: set 2
F fit (CVX) = 0.9967476173781533
F fit (MLE) = 0.9943835079887965
Random single-qubit unitaries: set 3
F fit (CVX) = 0.9998905315786057
F fit (MLE) = 0.997710169842333
Random single-qubit unitaries: set 4
F fit (CVX) = 0.9997560394298681
F fit (MLE) = 0.9963310432938237


## 5-Qubit Bell State

In [11]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, Aer.get_backend('statevector_simulator'))
psi_bell5 = job.result().get_statevector(bell5)

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = tomo.state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, Aer.get_backend('qasm_simulator'), shots=5000)

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_bell5 = tomo.tomography_data(job.result(), qst_bell5)
probs_bell5, basis_matrix_bell5 = tomo.fitter_data(tomo_counts_bell5)
print('Time taken:', time.time() - t)

Time taken: 204.00656533241272


In [12]:
t = time.time()
rho_mle_bell5 = tomo.state_mle_fit(probs_bell5, basis_matrix_bell5)
print('MLE Reconstruction')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_mle_bell5))

MLE Reconstruction
Time taken: 3.31719708442688
Fit Fidelity: 0.9934583753879618


In [13]:
t = time.time()
rho_sdp_bell5 = tomo.state_cvx_fit(probs_bell5, basis_matrix_bell5, solver='CVXOPT')
print('SDP Reconstruction')
print('Time taken:', time.time() - t)
print('Fidelity:', state_fidelity(psi_bell5, rho_sdp_bell5))

SDP Reconstruction
Time taken: 48.56195878982544
Fidelity: 0.999928063990402
